In [1]:
import pyodbc
import numpy as np
import pandas as pd

In [2]:
sql_driver = 'DRIVER={ODBC Driver 13 for SQL Server};'
sql_server = 'SERVER=sage;'
sql_db = 'DATABASE=BUYS;'
sql_UID = 'Trusted_Connection=yes;'

cnxn = pyodbc.connect(sql_driver + sql_server + sql_db + sql_UID)

def fetch_data(q, cnxn):
    df = pd.read_sql(sql=q, con=cnxn)
    return df

In [3]:
import pyodbc
import numpy as np
import pandas as pd

sql_driver = 'DRIVER={ODBC Driver 13 for SQL Server};'
sql_server = 'SERVER=sage;'
sql_db = 'DATABASE=BUYS;'
sql_UID = 'Trusted_Connection=yes;'

cnxn = pyodbc.connect(sql_driver + sql_server + sql_db + sql_UID)

def fetch_data(q, cnxn):
    df = pd.read_sql(sql=q, con=cnxn)
    return df

In [4]:
query_algcomp = f'''

SET NOCOUNT ON
SET ANSI_WARNINGS OFF
DECLARE @StartDate DATE = '9/1/2019'
DECLARE @EndDate DATE = '10/1/2019'
DECLARE @LastR4GenDate DATE
DECLARE @LastR41GenDate DATE


SELECT 
	@LastR4GenDate = MAX(ba4.Date_Generated)
FROM Buy_Analytics..BuyAlgorithm_V1_R4 ba4

SELECT 
	@LastR41GenDate = MAX(ba41.Date_Generated)
FROM Buy_Analytics..BuyAlgorithm_V1_R41 ba41

SELECT 
	ba4.CatalogID,
	MIN(ba4.Chain_SuggestedOffer) [Chain_SuggestedOffer],
	MIN(ba4.Chain_Buy_Offer_Pct) [Chain_SuggestedOfferPct]
INTO #ba4ChainOffers
FROM Buy_Analytics..BuyAlgorithm_V1_R4 ba4
WHERE ba4.Date_Generated = @LastR4GenDate
GROUP BY ba4.CatalogID

SELECT 
	ba41.CatalogID,
	MIN(ba41.Chain_SuggestedOffer) [Chain_SuggestedOffer],
	MIN(ba41.Chain_Buy_Offer_Pct) [Chain_SuggestedOfferPct]
INTO #ba41ChainOffers
FROM Buy_Analytics..BuyAlgorithm_V1_R41 ba41
WHERE ba41.Date_Generated = @LastR41GenDate
GROUP BY ba41.CatalogID


SELECT 
	bbh.LocationNo,
	bbh.BuyBinNo,
	bbi.ItemLineNo,
	bbi.CatalogID,
	CASE 
		WHEN t.binding IN ('Mass Market Paperback', 'Audio CD', 'CD') 
		THEN t.binding 
		ELSE 'General' 
		END [CatalogBinding],
	bbi.SuggestedOffer,
	bbi.Quantity,
	ISNULL(lba4.Location_SuggestedOffer, cba4.Chain_SuggestedOffer) [r40_SuggestedOffer],
	ISNULL(lba4.Location_Buy_Offer_Pct, cba4.Chain_SuggestedOfferPct) [r40_SuggestedOfferPct],
	ISNULL(lba41.Location_SuggestedOffer, cba41.Chain_SuggestedOffer) [r41_SuggestedOffer],
	ISNULL(lba41.Location_Buy_Offer_Pct, cba41.Chain_SuggestedOfferPct) [r41_SuggestedOfferPct],
	ISNULL(lbt42.BuyOfferPct * ISNULL(adl.Avg_Sale_Price, adc.Avg_Sale_Price), cbt42.BuyOfferPct * adc.Avg_Sale_Price) [r42_SuggestedOffer],
	ISNULL(lbt42.BuyOfferPct, cbt42.BuyOfferPct) [r42_SuggestedOfferPct]
FROM BUYS..BuyBinHeader bbh
	INNER JOIN BUYS..BuyBinItems bbi
		ON bbh.BuyBinNo = bbi.BuyBinNo
		AND bbh.LocationNo = bbi.LocationNo
	INNER JOIN Sandbox..LocBuyAlgorithms lba
		ON bbh.LocationNo = lba.LocationNo
		AND lba.VersionNo = 'v1.r4'
	INNER JOIN Catalog..titles t
		ON bbi.CatalogID = t.catalogId
	LEFT OUTER JOIN Buy_Analytics..BuyAlgorithm_V1_R4 lba4
		ON bbi.CatalogID = lba4.CatalogID
		AND lba4.Date_Generated = @LastR4GenDate
		AND bbi.LocationNo = lba4.LocationNo
	LEFT OUTER JOIN Buy_Analytics..BuyAlgorithm_V1_R41 lba41
		ON bbi.CatalogID = lba41.CatalogID
		AND lba41.Date_Generated = @LastR41GenDate
		AND bbi.LocationNo = lba41.LocationNo
	LEFT OUTER JOIN #ba4ChainOffers cba4
		ON bbi.CatalogID = cba4.CatalogID
	LEFT OUTER JOIN #ba41ChainOffers cba41
		ON bbi.CatalogID = cba41.CatalogID
	LEFT OUTER JOIN Buy_Analytics..BuyAlgorithm_AggregateData_Chain adc
		ON bbi.CatalogID = adc.CatalogID
	LEFT OUTER JOIN Buy_Analytics..BuyAlgorithm_AggregateData_Location adl
		ON bbi.LocationNo = adl.LocationNo
		AND bbi.CatalogID = adl.CatalogID
	LEFT OUTER JOIN Sandbox..AccumulatedDaysOnShelf_BuyTable_V1_R42 lbt42
		ON	(adl.Total_Accumulated_Days_With_Trash_Penalty / (adl.Total_Sold + 1)) > lbt42.AccDaysRangeFrom
		AND (adl.Total_Accumulated_Days_With_Trash_Penalty / (adl.Total_Sold + 1)) <= lbt42.AccDaysRangeTo
		AND (CASE 
				WHEN t.binding IN ('Mass Market Paperback', 'Audio CD', 'CD') 
				THEN t.binding 
				ELSE 'General' 
				END) = lbt42.CatalogBinding
	LEFT OUTER JOIN Sandbox..AccumulatedDaysOnShelf_BuyTable_V1_R42 cbt42
		ON	(adc.Total_Accumulated_Days_With_Trash_Penalty / (adc.Total_Sold + 1)) > cbt42.AccDaysRangeFrom
		AND (adc.Total_Accumulated_Days_With_Trash_Penalty / (adc.Total_Sold + 1)) <= cbt42.AccDaysRangeTo
		AND (CASE 
				WHEN t.binding IN ('Mass Market Paperback', 'Audio CD', 'CD') 
				THEN t.binding 
				ELSE 'General' 
				END) = cbt42.CatalogBinding
WHERE bbi.Scoring_ID IS NOT NULL
	AND bbh.CreateTime > @StartDate
	AND bbh.CreateTime < @EndDate
	AND bbh.StatusCode = 1
	AND bbi.StatusCode = 1
	AND bbi.Quantity > 0
ORDER BY bbh.LocationNo, bbh.BuyBinNo


DROP TABLE #ba4ChainOffers
DROP TABLE #ba41ChainOffers
'''

In [5]:
query = f'''
SET NOCOUNT ON
SET ANSI_WARNINGS OFF

DECLARE @StartDate DATE = '9/1/2018'
DECLARE @EndDate DATE = '10/1/2019'
DECLARE @LastR4ChainGenDate DATE
DECLARE @LastR4LocGenDate DATE

SELECT 
	@LastR4ChainGenDate = MAX(ba.Date_Generated)
FROM Buy_Analytics.dbo.BuyAlgorithm_V1_R4 ba


SELECT 
	ba.CatalogID,
	MIN(ba.Chain_Buy_Offer_Pct) [chain_BuyOfferPct],
	MIN(ba.Chain_SuggestedOffer) [chain_BuyOfferAmt]
INTO #ChainSuggestedOffers
FROM Buy_Analytics.dbo.BuyAlgorithm_V1_R4 ba
WHERE ba.Date_Generated = @LastR4ChainGenDate
GROUP BY ba.CatalogID


SELECT DISTINCT
	adc.CatalogID,
	bt.CatalogBinding,
	adc.Total_Item_Count,
	adc.Total_Accumulated_Days_With_Trash_Penalty,
	adc.Avg_Sale_Price,
	adc.Total_Sold,
	bt.BuyOfferPct [R4BuyOfferPct],
	bts.BuyOfferPct [R4sBuyOfferPct],
	adc.Total_Accumulated_Days_With_Trash_Penalty / adc.Total_Item_Count [avg_Item_Acc_Days],
	adc.Total_Accumulated_Days_With_Trash_Penalty / (adc.Total_Sold + 1) [avg_Sold_Acc_Days],
	CAST(adc.Avg_Sale_Price * bt.BuyOfferPct AS DECIMAL(19,2)) [R4BuyOfferAmt],
	CAST(adc.Avg_Sale_Price * bts.BuyOfferPct AS DECIMAL(19,2)) [R4sBuyOfferAmt],
	cso.chain_BuyOfferPct [chain_R4BuyOfferPct],
	cso.chain_BuyOfferAmt [chain_R4BuyOfferAmt]
INTO #ChainCalculatedOffers
FROM Buy_Analytics.dbo.BuyAlgorithm_AggregateData_Chain adc
	INNER JOIN Catalog..titles t
		ON adc.CatalogID = t.catalogId
	INNER JOIN Buy_Analytics..AccumulatedDaysOnShelf_BuyTable_V1_R4 bt
		ON (CASE
				WHEN t.binding IN ('Audio CD', 'CD', 'Mass Market Paperback')
				THEN t.binding
				ELSE 'General'
				END) = bt.CatalogBinding
		AND CAST((adc.Total_Accumulated_Days_With_Trash_Penalty / adc.Total_Item_Count) AS DECIMAL(19, 2)) > bt.AccDaysRangeFrom
		AND CAST((adc.Total_Accumulated_Days_With_Trash_Penalty / adc.Total_Item_Count) AS DECIMAL(19, 2)) <= bt.AccDaysRangeTo
	INNER JOIN Sandbox..AccumulatedDaysOnShelf_BuyTable_V1_R42 bts
		ON (CASE
				WHEN t.binding IN ('Audio CD', 'CD', 'Mass Market Paperback')
				THEN t.binding
				ELSE 'General'
				END) = bts.CatalogBinding
		AND CAST((adc.Total_Accumulated_Days_With_Trash_Penalty / (adc.Total_Sold + 1)) AS DECIMAL(19, 2)) > bts.AccDaysRangeFrom
		AND CAST((adc.Total_Accumulated_Days_With_Trash_Penalty / (adc.Total_Sold + 1)) AS DECIMAL(19, 2)) <= bts.AccDaysRangeTo
	INNER JOIN #ChainSuggestedOffers cso
		ON adc.CatalogID = cso.CatalogID
WHERE adc.Insert_Date > '10/24/19' AND adc.Insert_Date < '10/26/19'

SELECT 
	bbi.CatalogID,
	COUNT(bbi.CatalogID) [qty_Purchased]
INTO #ChainQtyPurchased
FROM BUYS..BuyBinItems bbi
	INNER JOIN Sandbox..LocBuyAlgorithms lba
		ON bbi.LocationNo = lba.LocationNo
	INNER JOIN Catalog..titles t
		ON bbi.CatalogID = t.catalogId
WHERE 
		bbi.StatusCode = 1
	AND bbi.CreateTime >= @StartDate
	AND bbi.CreateTime < @EndDate
	AND bbi.Quantity > 0
GROUP BY bbi.CatalogID

SELECT
	cqp.CatalogID,
    co.CatalogBinding,
	cqp.qty_Purchased,
	co.R4BuyOfferAmt,
	co.R4sBuyOfferAmt,
	co.R4BuyOfferPct,
	co.R4sBuyOfferPct,
	co.Total_Accumulated_Days_With_Trash_Penalty,
	co.Total_Item_Count,
	co.Total_Sold,
	co.Avg_Sale_Price,
	co.avg_Item_Acc_Days,
	co.avg_Sold_Acc_Days
FROM #ChainQtyPurchased cqp
	INNER JOIN #ChainCalculatedOffers co
		ON cqp.CatalogID = co.CatalogID

DROP TABLE #ChainSuggestedOffers
DROP TABLE #ChainCalculatedOffers
DROP TABLE #ChainQtyPurchased
'''

In [6]:
df  = fetch_data(query, cnxn)
df.to_csv('./BuyOffersByQtyPurchased.csv')

In [7]:
df_ac  = fetch_data(query_algcomp, cnxn)
df_ac.to_csv('./BuyOffersAlgComp.csv')